In [ ]:
from __future__ import division, print_function, absolute_import
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=False) 

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#type(mnist)

In [ ]:
#mnist.train.images

In [ ]:
#import matplotlib.pyplot as plt

In [ ]:
#mnist.train.images.shape

In [ ]:
single_img = mnist.train.images[1].reshape(28,28)

In [ ]:
plt.imshow(single_img)

In [ ]:
plt.imshow(single_img, cmap= 'gist_gray') # to see image in grey

In [ ]:
# Training parameter
learning_rate = 0.001
num_steps= 2000
batch_size= 120

# network paramter
num_input = 28*28  # MNIST data input(img shape : 28 * 28)
num_classes= 10  # MNIST total classes 0 -9 digits
dropout= 0.25  # dropout , probability to drop  a unit

In [ ]:
# create a neural network
def conv_net(x_dict, n_classes,dropout, reuse, is_training):
    
    # defining scope of reuse the variable
    with tf.variable_scope('convNet', reuse= reuse):
        # TF Estimator input is a dict, in case of multiple inputs
        x= x_dict['images']
        
        # mnist data is 1D input that is 784 (28 * 28 pixels)
        # Reshape the match picture format(Height * Width * channel)
        # Tensor input become 4D: [Batch size, Height, Width, channel]
        x= tf.reshape(x, [-1,28,28,1])
        
        # convolution layer with 32 filter and  kernal size of 5 
        conv1 = tf.layers.conv2d(x ,32, 5, activation = tf.nn.relu)
        # maxpooling (down -sampling) with  strides of 2  and kernel size of 2
        conv1 = tf.layers.max_pooling2d(conv1, 2,2)
        
        # convolution layer with size of 64 and kernel size of 3
        conv2= tf.layers.conv2d(conv1, 64, 3, activation= tf.nn.relu)
        # maxpooling ( down sampling ) with strides of 2 and kernel size of 2
        conv2 = tf.layers.max_pooling2d(conv2, 2,2)
        
        # flatten the into 1d for fully connected layer
        fc1 = tf.contrib.layers.flatten(conv2)
        
        # fully connected layers
        fc1= tf.layers.dense(fc1,1024)
        
        # apply dropout
        fc1 = tf.layers.dropout(fc1,rate=dropout, training= is_training)
        
        # output layers prediction 
        out = tf.layers.dense(fc1, n_classes)
        
    return out



In [ ]:
# define model fuction
def model_fun( features, labels, mode):
    # build the neural network
    # Dropout have 2 different behavior  at training and prediction time , 
    # we need to create 2 distinct computation graph with same weights
    
    logit_train = conv_net(features, num_classes, dropout, reuse=False, is_training=True)
    logit_test = conv_net(features, num_classes, dropout, reuse=True, is_training=False)
    
    # prediction
    pred_classes = tf.argmax(logit_test, axis=1)
    pred_proba= tf.nn.softmax(logit_test)
    
    # if prediction mode
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes) 
    
    # define loss and optimizer
    loss_op = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(
            logits= logit_train, 
            labels= tf.cast(labels, dtype= tf.int32)))
    
    optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    # evaluate accuracy of model
    acc_op= tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    
    estim_spacs = tf.estimator.EstimatorSpec( mode = mode,
                                            predictions=pred_classes,
                                            loss= loss_op,
                                            train_op= train_op,
                                            eval_metric_ops= {'accuracy':acc_op})
    return estim_spacs
    


In [ ]:
# Build the Estimator
model = tf.estimator.Estimator(model_fn)

In [ ]:
# define input function for training
input_fn = tf.estimator.inputs.numpy_input_fn(
            x= {'images':mnist.train.images}, y =mnist.train.labels,
            batch_size=batch_size, num_epochs=None, shuffle= True)

model.train(input_fn, steps=num_steps)



In [ ]:
# Predict single images
n_images = 10
# Get images from test set
test_images = mnist.test.images[:n_images]
# Prepare the input data
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': test_images}, shuffle=False)
# Use the model to predict the images class
preds = list(model.predict(input_fn))

# Display
for i in range(n_images):
    plt.imshow(np.reshape(test_images[i], [28, 28]), cmap='gray')
    plt.show()
    print("Model prediction:", preds[i])